**UTS TEMU KEMBALI INFORMASI**

Nama dan NIM Anggota 1: Thyara Mahadewi - 202110370311069

Nama dan NIM Anggota 2: Indri Reghina Putri - 202110370311096

## Library

In [ ]:
!pip install Sastrawi

In [ ]:
# Import SastrawiStemmer

In [114]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## Load data

In [115]:
url = 'https://raw.githubusercontent.com/nanajem1/Dataset-Narkotika_069_096/main/Overview/Overview.xlsx'
df = pd.read_excel(url)


In [116]:
df.head()

,NO,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan
0,1,13/Pid.Sus/2023/PN Jkt.Tim,PN Jakarta Timur,1.\t1 (satu kantong plastic klip bening berisi...,Menyatakan terdakwa DWI RADITYO ALS AMBON BIN ...
1,2,40 /PID.SUS/2023/PN.JKT.TIM.,PN Jakarta Timur,1.\t2(dua)buahkotakStainlessyangdidalamnyaberi...,Menyatakan Terdakwa EKI MAI HENDRA Bin HUSNI t...
2,3,41/Pid.Sus/2023/PN Jkt.Tim,PN Jakarta Timur,1.\t2 (dua) buah kotak stainles yang didalamny...,Menyatakan Terdakwa DANI ROHIMAT bin TATA AJAN...
3,4,42 /PID.SUS/2023/PN.JKT.TIM,PN Jakarta Timur,1.\t2 (dua) buah kotak stainles yang didalamny...,Menyatakan Terdakwa IMAM NURJAYA bin YUNUS BLE...
4,5,49/Pid.Sus/2023/PN Jkt.Tim,PN Jakarta Timur,1) 1 (satu) buah dompet warna merah berisi 1 (...,Menyatakan Terdakwa ROGEL SISCO PENTURY alias ...


## EDA

In [128]:
df.describe()

,NO
count,50.00000
mean,25.50000
std,14.57738
min,1.00000
25%,13.25000
50%,25.50000
75%,37.75000
max,50.00000


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NO                 50 non-null     int64 
 1   No Putusan         50 non-null     object
 2   Lembaga Peradilan  50 non-null     object
 3   Barang Bukti       50 non-null     object
 4   Amar Putusan       50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [118]:
df.isnull().sum()

,0
NO,0
No Putusan,0
Lembaga Peradilan,0
Barang Bukti,0
Amar Putusan,0


## Pre - Processing

In [119]:
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [120]:
def clean_data(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(text.split())
    return text


In [121]:
def tokenize(text):
    return word_tokenize(text)


In [76]:
# def remove_stopwords(tokens):
#     # Menghapus stop words
#     return [word for word in tokens if word not in stop_words]


In [122]:
def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]


In [123]:
def preprocess_text(text):
    cleaned_text = clean_data(text)
    tokens = tokenize(cleaned_text)
    stemmed_tokens = stem_tokens(tokens)
    return ' '.join(stemmed_tokens)

df['Barang Bukti'] = df['Barang Bukti'].apply(preprocess_text)
df['Amar Putusan'] = df['Amar Putusan'].apply(preprocess_text)
df['cleaning'] = df['Barang Bukti'] + ' ' + df['Amar Putusan']

In [124]:
df[['NO', 'No Putusan', 'Lembaga Peradilan', 'Barang Bukti', 'Amar Putusan']].head()


,NO,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan
0,1,13/Pid.Sus/2023/PN Jkt.Tim,PN Jakarta Timur,1 1 satu kantong plastic klip bening risi 2 du...,nyata dakwa dwi radityo als ambon bin lardanis...
1,2,40 /PID.SUS/2023/PN.JKT.TIM.,PN Jakarta Timur,1 2duabuahkotakstainlessyangdidalamnyaberisika...,nyata dakwa eki mai hendra bin husni telah buk...
2,3,41/Pid.Sus/2023/PN Jkt.Tim,PN Jakarta Timur,1 2 dua buah kotak stainles yang dalam risi 41...,nyata dakwa dan rohimat bin tata ajan telah bu...
3,4,42 /PID.SUS/2023/PN.JKT.TIM,PN Jakarta Timur,1 2 dua buah kotak stainles yang dalam risi 41...,nyata dakwa imam nurjaya bin yunus blegur tela...
4,5,49/Pid.Sus/2023/PN Jkt.Tim,PN Jakarta Timur,1 1 satu buah dompet warna merah isi 1 satu bu...,nyata dakwa rogel sisco pentury alias chiko an...


## Indexing

In [125]:
from collections import defaultdict

# Fungsi indexing yang lebih komprehensif
def create_inverted_index(df, text_column):
    inverted_index = defaultdict(set)  # Menggunakan set untuk menghindari duplikasi

    for i, row in df.iterrows():
        processed_text = preprocess_text(row[text_column])
        for word in processed_text.split():
            inverted_index[word].add(i)

    return inverted_index

inverted_index = create_inverted_index(df, 'cleaning')


In [126]:
def search(query):
    query = query.lower()
    query_tokens = [stemmer.stem(word) for word in query.split() if word not in stop_words]

    # Mulai pencarian berdasarkan indeks
    if not query_tokens:
        return pd.DataFrame()

    # Mengambil indeks dari kata pertama dalam query
    result = inverted_index[query_tokens[0]] if query_tokens[0] in inverted_index else set()
    for token in query_tokens[1:]:
        if token in inverted_index:
            result.intersection_update(inverted_index[token])
        else:
            result = set()
            break

    # Mengambil baris yang valid dari DataFrame
    valid_indices = [i for i in result if i in df.index]
    return df.loc[valid_indices] if valid_indices else pd.DataFrame()


In [127]:
df_result = search("aditia")
df_result.head()


,NO,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan,cleaning
27,28,280/Pid.Sus/2023/PN JKT.TIM,PN Jakarta Timur,1 1 satu bungkus plastik bening risi sabu bera...,nyata dakwa aditia alias erlangga bins amsular...,1 1 satu bungkus plastik bening risi sabu bera...
